In [99]:
import os
import re


In [100]:
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [109]:
def train_val_split(sent_list,split_percent):
    split_val = int(len(sent_list)*split_percent)
    train_list = sent_list[:split_val]
    val_list = sent_list[split_val:]
    
    return train_list, val_list

def remove_sgm(sent):
    return re.sub('<seg id="[0-9]+">|<\/seg>|\\n', '', sent)

def output_list_2_txt(my_list,output_path):
    with open(output_path,"w") as f:
        for sent in my_list:
            f.write("%s\n" % sent)
        f.flush()
        
def filter_sentences(en_list,de_list,min_length, max_length):
    new_en_list = []
    new_de_list = []
    sent_dropped = 0

    for i in range(len(en_list)):
        if len(en_list[i]) >= min_length and len(en_list[i]) <= max_length:
            if len(de_list[i]) >= min_length and len(de_list[i]) <= max_length:
                new_en_list.append(en_list[i])
                new_de_list.append(de_list[i])
            else:
                sent_dropped +=1     
        else:
            sent_dropped +=1

    if len(new_en_list) == len(new_de_list):
        print("Length of both lists match {}".format(len(new_de_list)))
    else:
        print("Somethings gone terribly wrong!!")
    return new_en_list, new_de_list 

In [102]:
from mosestokenizer import *

In [103]:
tokenizer = MosesTokenizer('en')
detokenizer = MosesDetokenizer('en')


In [104]:
f = open("/home/jupyter/Getting_New_Data/paracrawl_data/test/paracrawl-ende-src.en.sgm", "r")
en_list = f.readlines()
#Remove non applicable lines
en_list = en_list[3:-3]
print(en_list[0])
print(en_list[-1])

<seg id="0">But it was. Specifically, what was broken was the story assigning value to mortgage-backed securities and other derivatives based on unrepayable loans.</seg>

<seg id="99999">I was not even out rightly afraid.</seg>



In [105]:
f = open("/home/jupyter/Getting_New_Data/paracrawl_data/paracrawl-ende-src.hyp.detok.de", "r")
de_list = f.readlines()
print(de_list[0])
print(de_list[-1])

Was jedoch konkret gebrochen wurde, war die Geschichte, die hypothekenbesicherten Wertpapieren und anderen Derivaten auf der Grundlage nicht rückzahlbarer Kredite Wert zuweist.

Ich hatte nicht einmal zu Recht Angst.



In [106]:
#Remove SGM formatting
en_list = list(map(remove_sgm,en_list))
de_list = list(map(remove_sgm,de_list))
print("Length of en list:{}".format(len(en_list)))
print(en_list[0])
print(de_list[0])
print(en_list[-1])
print(de_list[-1])

Length of en list:100000
But it was. Specifically, what was broken was the story assigning value to mortgage-backed securities and other derivatives based on unrepayable loans.
Was jedoch konkret gebrochen wurde, war die Geschichte, die hypothekenbesicherten Wertpapieren und anderen Derivaten auf der Grundlage nicht rückzahlbarer Kredite Wert zuweist.
I was not even out rightly afraid.
Ich hatte nicht einmal zu Recht Angst.


In [107]:
#Tokenise Sentences
en_list_token = list(map(tokenizer,en_list))
de_list_token = list(map(tokenizer,de_list))
print("Length of tokenised en list:{}".format(len(en_list_token)))
print(en_list_token[0])
print(de_list_token[0])
print(en_list_token[-1])
print(de_list_token[-1])

Length of tokenised en list:100000
['But', 'it', 'was', '.', 'Specifically', ',', 'what', 'was', 'broken', 'was', 'the', 'story', 'assigning', 'value', 'to', 'mortgage', '@-@', 'backed', 'securities', 'and', 'other', 'derivatives', 'based', 'on', 'unrepayable', 'loans', '.']
['Was', 'jedoch', 'konkret', 'gebrochen', 'wurde', ',', 'war', 'die', 'Geschichte', ',', 'die', 'hypothekenbesicherten', 'Wertpapieren', 'und', 'anderen', 'Derivaten', 'auf', 'der', 'Grundlage', 'nicht', 'rückzahlbarer', 'Kredite', 'Wert', 'zuweist', '.']
['I', 'was', 'not', 'even', 'out', 'rightly', 'afraid', '.']
['Ich', 'hatte', 'nicht', 'einmal', 'zu', 'Recht', 'Angst', '.']


In [110]:
max_length = 100
min_length = 3
sent_dropped = 0
new_en_list, new_de_list = filter_sentences(en_list_token,de_list_token,min_length, max_length)
    

Length of both lists match 99673


In [111]:
#De-Tokenise EN Sentences
en_list = list(map(detokenizer,new_en_list))
de_list = list(map(detokenizer,new_de_list))
#De-Tokenise DE Sentences
de_list = list(map(detokenizer,new_de_list))
print(en_list[0])
print(de_list[0])
print(en_list[-1])
print(de_list[-1])

But it was. Specifically, what was broken was the story assigning value to mortgage-backed securities and other derivatives based on unrepayable loans.
I was not even out rightly afraid.
Was jedoch konkret gebrochen wurde, war die Geschichte, die hypothekenbesicherten Wertpapieren und anderen Derivaten auf der Grundlage nicht rückzahlbarer Kredite Wert zuweist.
Ich hatte nicht einmal zu Recht Angst.


In [97]:
en_list = en_list_token
de_list = de_list_token

In [98]:
biggest =  int(0)
smallest = int(0)
for i in range(len(en_list)):
    if len(en_list[i]) >= biggest:
        biggest = len(en_list[i])
        
    if len(de_list[i]) >= biggest:
        biggest = len(de_list[i])
    
    if len(en_list[i]) <= smallest:
        smallest = len(en_list[i])
    
    if len(de_list[i]) <= smallest:
        smallest = len(de_list[i])

print("Biggest value {}".format(biggest))
print("Smallest value {}".format(smallest))

Biggest value 250
Smallest value 0


In [129]:
split_p = 0.985
de_train, de_val = train_val_split(de_list,split_p)
en_train, en_val = train_val_split(en_list,split_p)
print('length of training dataset :{}, length of validation dataset:{}'.format(len(de_train),len(de_val)))

length of training dataset :98177, length of validation dataset:1496


In [130]:
output_dir = '/home/jupyter/Getting_New_Data/NewBasicModel/dataset'
output_de_train = output_dir+'/tgt-train.txt'
output_de_val = output_dir+'/tgt-val.txt'
output_en_train = output_dir+'/src-train.txt'
output_en_val = output_dir+'/src-val.txt'

output_list_2_txt(de_list,output_de_train)
output_list_2_txt(de_list,output_de_val)
output_list_2_txt(en_list,output_en_train)
output_list_2_txt(en_list,output_en_val)